In [ ]:
import pandas as pd
import re
import numpy as np
def read_vcf(vcf_file, separator = '\t'):
    head_line = recognize_vcf_head(vcf_file)
    vcf = pd.read_table(vcf_file, header = head_line, sep = separator)
    vcf = vcf.set_index('ID')
    vcf = vcf.drop(['QUAL', 'FILTER', 'INFO', 'FORMAT'], axis = 1)
    locis = vcf.columns[4:]
    for loci in locis:
        vcf[loci] = vcf[loci].str.extract(r'([\d\.]/[\d\.]):*')
    return vcf
def vcf_to_genepop(vcf_file, popmap,genepop_name):

    pop_map_dict = read_pop_map(popmap)
    vcf = read_vcf(vcf_file)
    locis = vcf.columns[4:]
    for loci in locis:
        rag = vcf['REF'] + vcf['ALT'] + vcf[loci]
        vcf[loci] = rag.apply(change_num_to_base)

    genepop = vcf
    chrom_pos = vcf['#CHROM'].astype(str) + '_' +(vcf['POS']-1).astype(str) 
    genepop.insert(0,'CHROM_POS', chrom_pos)
    genepop = genepop.drop(['REF','ALT','#CHROM', 'POS' ], axis = 1)
    
    for loci in locis:
        genepop[loci] = genepop[loci].apply(change_base_to_genepop_number)
    
    genepop = genepop.T
    genepop.columns = d.iloc[0]
    genepop.head(1).to_csv(genepop_name, header = False, index  = False, sep  = '\t')
    for pop in pop_map_dict:
        with open(genepop_name,'a') as f:
            f.write('pop' + '\n')
            f.close()
        genepop.loc[pop_map_dict[pop]].to_csv(genepop_name, 
                                               header = False, 
                                               index  = True, 
                                               sep  = '\t', 
                                               mode = 'a')
    print('File was generated')
    return genepop


def change_num_to_base(ref_alt_gen):
    ref = ref_alt_gen[0]
    alt = ref_alt_gen[1]
    ref_alt_gen = ref_alt_gen.replace('0',ref)
    ref_alt_gen = ref_alt_gen.replace('1',alt)
    ref_alt_gen = ref_alt_gen.replace('/','')
    return ref_alt_gen[2:]

def change_base_to_genepop_number(genotype):
    bases_dict = {'A':'01','T':'02','G':'03','C':'04','.':'00'}
    genepop_code = ''
    for base in genotype:
        genepop_code += bases_dict[base]
    return genepop_code
        
    
def read_pop_map(popmap, separator = '\t'):
    popdf = pd.read_table(popmap, sep = separator, header = None)
    popdf = popdf.drop_duplicates()
    unique_pops = popdf[1].unique()
    pop_dict = {}
    for unique_pop in unique_pops:
        pop_dict[unique_pop] =  popdf[popdf[1] == unique_pop][0].values.tolist()
    return pop_dict
def recognize_vcf_head(vcf_file):
    with open(vcf_file) as file:
        line = file.readline()
        i = 0
        while 'CHROM' not in line:
            line = file.readline()
            i += 1
    return i


In [ ]:
popmap = 'data_pop_map.txt'
b = read_pop_map(popmap)

In [ ]:
d = vcf_to_genepop('Xr-567.vcf','data_pop_map.txt','test1')

In [ ]:
read_pop_map('data_pop_map.txt')

In [ ]:
d.columns = d.iloc[0]
d.loc['CHROM_POS'].to_csv('test', header = False, index  = False )

In [ ]:
d.loc['CHROM_POS']

In [ ]:
d.head(1).to_csv('test', header = False, index  = True, sep  = '\t')

In [1]:
import pandas as pd
import re
import numpy as np
import csv
def read_vcf(vcf_file, separator = '\t'):
    head_line = recognize_vcf_head(vcf_file)
    vcf = pd.read_table(vcf_file, header = head_line, sep = separator)
    vcf = vcf.set_index('ID')
    vcf = vcf.drop(['QUAL', 'FILTER', 'INFO', 'FORMAT'], axis = 1)
    locis = vcf.columns[4:]
    for loci in locis:
        vcf[loci] = vcf[loci].str.extract(r'([\d\.]/[\d\.]):*')
    return vcf
def vcf_to_genepop(vcf_file,ped_name):
    
    map_name = ped_name.split('.')[0] + '.map'
    vcf = read_vcf(vcf_file)
    locis = vcf.columns[4:]
    for loci in locis:
        rag = vcf['REF'] + vcf['ALT'] + vcf[loci]
        vcf[loci] = rag.apply(change_num_to_base)

    genepop = vcf
    
    #Generate ped map file
    generate_ped_map(genepop[['#CHROM','POS']],map_name)
    
    chrom_pos = vcf['#CHROM'].astype(str) + '_' +(vcf['POS']-1).astype(str) 
    genepop.insert(0,'CHROM_POS', chrom_pos)
    genepop = genepop.drop(['REF','ALT','#CHROM', 'POS' ], axis = 1)
    
    genepop = genepop.T
    genepop.columns = genepop.iloc[0]
    
    #Change format to the ped format
    genepop = genepop.drop('CHROM_POS')
    genepop.insert(0,'CHROMPOS',genepop.index)
    
    zero_column = np.zeros(genepop.index.shape).astype(int)
    for i in range (4):
        genepop.insert(1,str(i),zero_column)
    genepop.to_csv(ped_name, header = False, sep = '\t')
    
    #The file add " for every genotype, so we open the file and delete them
    ped_file = open('ped_test')
    new_text = ped_file.read()
    new_text = new_text.replace('"','')
    ped_file.close()
    ped_file = open('ped_test','w')
    ped_file.write(new_text)
    ped_file.close()
    
    print('Ped and map ped files were generated')
    return genepop

def generate_ped_map(chrom_pos,map_name):
    zero_column = np.zeros(chrom_pos.index.shape).astype(int)
    combined_column = chrom_pos['#CHROM'].astype(str) + ':' +(chrom_pos['POS']-1).astype(str)
    chrom_pos.insert(1,'zero',zero_column)
    chrom_pos.insert(1,'combined',combined_column)
    chrom_pos.to_csv(map_name, header = False, sep  = '\t', index = False)
    
    
def change_num_to_base(ref_alt_gen):
    ref = ref_alt_gen[0]
    alt = ref_alt_gen[1]
    ref_alt_gen = ref_alt_gen.replace('0',ref)
    ref_alt_gen = ref_alt_gen.replace('1',alt)
    ref_alt_gen = ref_alt_gen.replace('/','\t')
    ref_alt_gen = ref_alt_gen.replace('.','0')
    return ref_alt_gen[2:]


def recognize_vcf_head(vcf_file):
    with open(vcf_file) as file:
        line = file.readline()
        i = 0
        while 'CHROM' not in line:
            line = file.readline()
            i += 1
    return i

In [2]:
a = vcf_to_genepop('Xr-567.vcf','ped_test')


Ped and map ped files were generated


# Fix vcf2stru bug

Bug source: 

The generated strucuture file is not readable for programs such as adegenet. This was cause the population column is label. adegenet recognize that label as a marker. 

To solve the problem we replace:

~~~
structure_file.insert(0,column = 'sample', value = sample_column)
~~~
by
~~~
structure_file.insert(0,column = '', value = sample_column)
~~~

In [3]:

import pandas as pd
def vcf_to_stru(vcf_file, pop_map ,stru_out, separator = '\t',):
    '''This function convert a vcf file to a struc file'''
    
    head_line = recognize_vcf_head(vcf_file)
    vcf = pd.read_table(vcf_file, header = head_line, sep = separator)
    vcf = vcf.set_index('ID')
    locis = vcf.columns[8:]
    stru = pd.DataFrame()

    for loci in locis:
        stru[loci] = vcf[loci].apply(get_GT)
    all1, all2 = separate_alleles(stru)
    all1['REF'] = vcf['REF'] 
    all1['ALT'] = vcf['ALT']
    
    all2['REF'] = vcf['REF'] 
    all2['ALT'] = vcf['ALT']
    
    all1 = replace_bases(all1, locis)
    all2 = replace_bases(all2, locis)
    all1 = all1.drop(['REF','ALT'], axis = 1)
    all2 = all2.drop(['REF','ALT'], axis = 1)
    structure_file = all1.T.iloc[[0,1]]
    structure_file = structure_file.drop(structure_file.index)

    for index in all1.T.index:
        i = 0
        structure_file.loc[str(i) + index] = all1.T.loc[index]
        i+= 1
        structure_file.loc[str(i) + index] = all2.T.loc[index]
    structure_file.index = [id[1:] for id in structure_file.index]
    
    pop_dict = read_pop_map(pop_map)
    sample_column = structure_file.index.to_series().map(pop_dict)
    structure_file.insert(0,column = '', value = sample_column)
    structure_file.to_csv(stru_out, sep = '\t')
    return structure_file

def read_pop_map(pop_map):
    pop_file = pd.read_table(pop_map, header = None, sep = '\t')
    pop_file = pop_file.drop_duplicates()
    pop_file = pop_file.set_index(0)
    pop_file[1].to_dict()
    return pop_file[1].to_dict()
    
    
def recognize_vcf_head(vcf_file):
    with open(vcf_file) as file:
        line = file.readline()
        i = 0
        while 'CHROM' not in line:
            line = file.readline()
            i += 1
    return i
def get_GT(loci):
    return loci[:3]
def drop_GT(GT):
    return GT[0]
def drop_get(GT):
    return GT[-1]

def separate_alleles(stru_file):
    all1 = pd.DataFrame()
    all2 = pd.DataFrame()
    for loci in stru_file.columns:
        all1[loci] = stru_file[loci].apply(drop_GT)
        all2[loci] = stru_file[loci].apply(drop_get)
    return all1, all2
def replace_bases(stru, locis):
    mut_stru_dict = {'.':0, 'A':1,'T':2,'G':3,'C':4}
    for loci in locis:
        stru.loc[stru[loci] == '0',loci] = stru[stru[loci] == '0']['REF']
        stru.loc[stru[loci] == '1',loci] = stru[stru[loci] == '1']['ALT']
        stru[loci] = stru[loci].map(mut_stru_dict)
    return stru

In [4]:
vcf_to_stru('Xr-567.vcf','data_pop_map.txt','generated_stru_fixed.stru')

ID,,43806:70,43978:60,44472:7,44672:60,44728:20,44876:14,45132:28,45154:78,45164:38,...,1404173:61,1404424:88,1408216:36,1410040:80,1418099:11,1422564:83,1422973:66,1428956:26,1437671:62,1474931:50
Xr_SBI_12-2,1,2,3,2,1,4,4,4,3,4,...,1,2,3,2,3,1,0,0,1,3
Xr_SBI_12-2,1,4,3,4,1,4,4,4,3,4,...,1,2,3,2,3,1,0,0,1,3
Xr_SBI_26-4,1,2,3,2,1,4,0,4,3,4,...,0,2,3,2,0,1,2,2,1,0
Xr_SBI_26-4,1,4,3,4,1,4,0,4,1,4,...,0,2,3,2,0,1,2,2,1,0
Xr_SBI_28-4,1,0,0,2,0,4,4,0,1,4,...,1,2,0,2,0,1,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xr_SNI_48-5,3,4,3,0,1,4,4,4,3,2,...,1,2,3,2,3,0,2,2,1,3
Xv_JTS_20-2,4,0,0,0,3,2,0,3,0,0,...,4,3,3,2,1,1,2,4,4,3
Xv_JTS_20-2,4,0,0,0,3,2,0,3,0,0,...,4,3,1,1,1,3,3,4,4,3
Xv_JTS_22-2,4,0,0,0,3,2,0,3,0,0,...,1,2,1,1,3,3,3,2,1,0
